# Assignment 3

In [5]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


#### Problem 1: Read the Data
Description:

#### Problem 2: Predict Percent Voting on al regressors
Description:

#### Problem 3: Formula Creating Function
Description:

#### Problem 4: R-Squared Computation Function
Description:

#### Problem 5: Find the best regressor
Description:

#### Problem 6: Pick the K-best regressors
Description:

#### Problem 7: Order all regressors in the best order
Description:


#### Problem 8: Plot the R-squared values as regressors are added in the best order.
Description:

#### Problem 9: Create a classication target.
Description

#### Problem 10: Create training and test sets.
Description:

#### Problem 11: Accuracy of classication
Description:

#### Problem 12: Plots and Reasons
Description: 